# Статистические методы. ДЗ2

***Автор: Лосевской Артём***

In [2]:
import numpy as np
import pandas as pd
from scipy import stats

## Задача 1

Случайная велчина $X$ - напряжение батарейки

Имеем 7 наблюдений 12,9; 11,6; 13,5; 13,9; 12,1; 11,9; 13,0.

- Тогда выборочное среднее $\overline{X}=\frac{12,9 + 11,6 + 13,5 + 13,9 + 12,1 + 11,9 + 13,0}{7}$

- Будем тестировать гипотезу $H_0: \mu_0 = E(X) = 12$ против $H_a: \mu_0 \neq 12$. 

Тестировать буду на 5% уровне значимости $(\alpha = 0.05)$; $p-value$ нужно считать двустороннее

- Тестируем с помощью t-теста: $T = \dfrac{(\overline{X}-\mu_0)\sqrt{n}}{s}, \; s = \sqrt{s^2}=\sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(X-\overline{X})}$, при этом случайная величина $T \sim t(n-1)$ (лемма Фишера)

*Note*: numpy считает $s = \sqrt{s^2}=\sqrt{\frac{1}{n}\sum_{i=1}^{n}(X-\overline{X})}$, поэтому $T = \dfrac{(\overline{X}-\mu_0)\sqrt{n-1}}{s}$

### Вручную

In [3]:
X_i = np.array([12.9, 11.6, 13.5, 13.9, 12.1, 11.9, 13.0])
n = len(X_i)
X_bar = np.sum(X_i) / n
mu_0 = 12

df = n - 1
t = stats.t(df) #t-распредление с соответствующим числом степеней свободы
s = X_i.std()
T = (X_bar - mu_0) * np.sqrt(n-1) / s
p = t.cdf(T)
print(f't-statistic = {T:.3f}, p-value = {min(2*(1-p),2*p):.3f}')

t-statistic = 2.158, p-value = 0.074


Выборочное среднее:

In [4]:
X_bar

12.700000000000001

### Функция

In [5]:
stats.ttest_1samp(X_i, mu_0)

TtestResult(statistic=2.1578017974978794, pvalue=0.07430190210562303, df=6)

### Результат

Итого получили $T=2.158 < T_{0.975}(6) = 2.447$ и $p-value > 0.025$

- Следовательно, не можем отвергнуть нулевую гипотезу. Принимаем, $H_0: \mu = 12$ (среднее напряжение действительно равно 12 В)

*P.S.* Важно смотреть на степени свободы: у нас $n$ мало, так что распределение не является приближенно стандартным нормальным.

*P.P.S.* Так как в условии не был оговорен уровень значимости, то проговорю, что и при $\alpha=10%$ не смогли бы отвергнуть

## Задача 2

Данные:

In [6]:
data = {
    'Образец': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Метод I': [23.1, 23.2, 26.5, 26.6, 27.1, 48.3, 40.5, 25.0, 38.4, 23.5],
    'Метод II': [22.7, 23.6, 27.1, 27.4, 27.4, 46.8, 40.4, 24.9, 38.1, 23.8]
}

df = pd.DataFrame(data)
print(df)

   Образец  Метод I  Метод II
0        1     23.1      22.7
1        2     23.2      23.6
2        3     26.5      27.1
3        4     26.6      27.4
4        5     27.1      27.4
5        6     48.3      46.8
6        7     40.5      40.4
7        8     25.0      24.9
8        9     38.4      38.1
9       10     23.5      23.8


In [7]:
# m1_mean = np.mean(data['Метод I'])
# m2_mean = np.mean(data['Метод II'])
# print(f'Выборочные средние {m1_mean} и {m2_mean} для методов I и II соответственно')

Поскольку работаем с одними и теми же образцами мяса (выборка одна, парные наблюдения), то для каждого из образцов найдём разницу в показаниях методов:

In [8]:
df['Разница'] = df['Метод II'] - df['Метод I']
print(df)

   Образец  Метод I  Метод II  Разница
0        1     23.1      22.7     -0.4
1        2     23.2      23.6      0.4
2        3     26.5      27.1      0.6
3        4     26.6      27.4      0.8
4        5     27.1      27.4      0.3
5        6     48.3      46.8     -1.5
6        7     40.5      40.4     -0.1
7        8     25.0      24.9     -0.1
8        9     38.4      38.1     -0.3
9       10     23.5      23.8      0.3


Теперь найдём выборочное среднее разницы, при этом разница показаний для нас и будет случайной величиной $X$:

$\overline{X}=\text{среднее разницы показаний}$

In [9]:
diff_mean = np.mean(df['Разница'])
print(f'{diff_mean:.16f}')

-0.0000000000000004


Тестировать можно таким образом:

- $H_0: E(X) = 0$ (в среднем методы дают одинаковые показатели)

- $H_a: E(X) \neq 0$ (в среднем один из методов даёт показания больше, чем другой)

То есть тут включаем $E(X) > 0$ (второй метод даёт показания больше) и $E(X) < 0$ (первый метод даёт показания больше)

Видно сходу, что для любого разумного уровня значимости нулевая гипотеза будет принята (т-статистика будет близка к нулю, p-value очень велико):

In [10]:
X_i = np.array(df['Разница'].tolist())
n = len(X_i)
X_bar = np.mean(X_i)
mu_0 = 0

df = n - 1
t = stats.t(df)
s = X_i.std()
T = (X_bar - mu_0) * np.sqrt(n-1) / s
p = t.cdf(T)
print(f't-statistic = {T:.3f}, p-value = {min(2*(1-p),2*p):.3f}')

t-statistic = -0.000, p-value = 1.000


In [11]:
stats.ttest_1samp(X_i, mu_0)

TtestResult(statistic=-1.7154884978678027e-15, pvalue=0.9999999999999987, df=9)

### Результат

Полученные результаты не позволяют сделать вывод о том, что один метод даёт в среднем более высокие показания, чем другой. 

Наши результаты свидетельствуют, что в среднем методы дают одинаковые показания

## Задача 3

$H_0: E(X) > E(Y)$, где $X$ - случайная величина, систолическое артериальное давление горожан, $Y$ - селян.

Имеем выборки из двух популяций объемов $n_X = 10$ горожан и $n_Y = 9$ селян.

Данные по горожанам: 132, 111, 119, 138, 200, 131, 138, 170, 159, 140.

Данные по селянам: 115, 190, 127, 155, 148, 121, 116, 121, 197.

Требуется протестировать $H_0$ при $\alpha = 0.05$.

- Гипотеза сложная, поэтому поступим иначе: будем тестировать $H_0: E(X) = E(Y)$ против $H_a: E(X) > E(Y)$

- $H_0: E(X) - E(Y) = 0$ против $H_a: E(X) - E(Y) > 0$

Выборки малы, но для решения такой задачи удобно сделать предположение об их нормальности, и тогда будем иметь распределения:

- $\bar{X} \sim N(E(X), \frac{\sigma^2_x}{n_x}), X \sim N(E(X), \sigma^2_x)$

- $\bar{Y} \sim N(E(Y), \frac{\sigma^2_y}{n_y}), Y \sim N(E(Y), \sigma^2_y)$

И тогда в силу свойств нормального распределения и независимости двух выборок (две разные популяции): $\bar{X}-\bar{Y} \sim N\left(E(X)-E(Y), \frac{\sigma^2_x}{n_x}+\frac{\sigma^2_y}{n_y}\right)$

В таком виде просто тестировали бы гипотезу о среднем случайной величины $\bar{X}-\bar{Y}$:

- $H_0: E(\bar{X}-\bar{Y}) = 0 \Leftrightarrow E(X) - E(Y) = 0$ против $H_a: E(X) - E(Y) > 0$

Итого нужный нам двухвыборочный т-критерий Стьюдента с независимыми выборками:

$T = \dfrac{\bar{X}-\bar{Y}}{\sqrt{\frac{s^2_x}{n_x}+\frac{s^2_y}{n_y}}}$

### Вручную

In [12]:
x_i = np.array([132, 111, 119, 138, 200, 131, 138, 170, 159, 140])
y_i = np.array([115, 190, 127, 155, 148, 121, 116, 121, 197])

n_x = len(x_i)
n_y = len(y_i)

x_bar = np.mean(x_i)
y_bar = np.mean(y_i)
s2_x = x_i.var()
s2_y = y_i.var()
T = (x_bar - y_bar) / np.sqrt(s2_x / n_x + s2_y / n_y)

k = (s2_x / n_x + s2_y / n_y) ** 2 / ((s2_x / n_x ) ** 2 / (n_x - 1) + (s2_y / n_y ) ** 2 / (n_y - 1)) #df

Округляем: $k=16$ 

In [13]:
t = stats.t(16) #t-распредление с соответствующим числом степеней свободы
F_T = t.cdf(T)
print(f't-statistic = {T:.3f}, p-value = {1-F_T:.3f}') #правосторонняя гипотеза

t-statistic = 0.037, p-value = 0.486


### Функция

С помощью scipy можно было бы тоже попробовать решить, но он подразумевает, что дисперсии выборок одинаковы, чего мы не можем утверждать.

In [14]:
stats.ttest_ind(x_i, y_i, equal_var=True, alternative='greater')

TtestResult(statistic=0.035165199049777565, pvalue=0.486178812200461, df=17.0)

### Результат

Тем не менее, значения получились очень близкие:

$0.49 = p-value > 0.05$, принимаем $H_0$.

**Ответ:** нет, на уровне значимости 5% считать верной выдвинутую гипотезу (жители городов имеют в среднем более высокое систолическое артериальное давление, чем жители сельской местности) нельзя.

## Задача 4

Известно, что 52% новорождённых детей — мальчики, а 48% — девочки. В случайной выборке из 5 000 человек в возрасте от 30 до 60 лет мужчин оказалось 2 500. Можно ли на уровне значимости 0,05 считать, основываясь на этих данных, что смертность среди мужчин и женщин одинакова?

**Решение**

Интерпретировать вопрос можно как равенство долей умерших мужчин и умерших женщин. Тогда, если смертность одинакова, то ожидаем, что в нашей случайной выборке будем иметь такое же соотношение среди живущих мужчин и женщин, как и среди новорождённых.

Другими словами, должны проверить, что доля $p$ живущих мужчин равна доле $p_0$ мужчин среди новорождённых.

$H_0: p = 0.52$

$H_a: p \neq 0.52$

Итак, имеем задачу на применение биномиального критерия (он же $z$-тест для одной пропорции):

- Число наблюдений $n=5000$

- Число успехов $m=2500$

$T=\frac{m-np_0}{\sqrt{np_0(1-p_0)}}$

Число наблюдений велико, при нулевой гипотезе в силу ЦПТ $T\sim Z(0,1)$

In [20]:
m = 2500
n = 5000
p_0 = 0.52
T = (m - n * p_0) / np.sqrt(n*p_0 * (1-p_0))
F_z = stats.norm.cdf(T)
print(f'z-statistic = {T:.3f}, p-value = {min(2*(1-F_z),2*F_z):.3f}') #двусторонняя гипотеза

z-statistic = -2.831, p-value = 0.005


### Результат

Итак, значение $z$-статистики $-2.831 < F_Z(\frac{\alpha}{2}) = F_Z(0.025)=-1.96$, значение $p-val$ очень мало: $0.005 < 0.05$.

Отвергаем гипотезу $H_0$. То есть на наших данных нельзя утверждать, что смертность среди мужчин и женщин одинакова.

## Задача 5

В первом эксперименте для сравнения двух средних X1=17, X2=16 применялся t-критерий Стьюдента, и эти различия оказались значимы (p = 0, 001).

Во втором исследовании, также при помощи t - критерия, сравнивались два средних X1=17, X2=36, и эти различия не значимы при p = 0,8. В чем может быть причина таких результатов, при условии, что нулевая и альтернативная гипотезы в обоих случаях одинаковы?

**Решение**

При любом разумном уровне значимости результаты не изменятся, а потому установленные уровни значимости (если разные) не являются причиной.

Среди причин можно выделить следующее:

1. Количество наблюдений (размеры выборок)
    
    В зависимости от количества наблюдений, мы получаем более или менее robust результаты тестов. Во каком-либо из тестов (скорее во втором) размер выборок мог оказаться слишком мал, чтобы свидетельствовать в пользу статистической значимости разницы средних. 

2. Принадлежность выборок разным популяциям (и, как следствие, разные параметры распределения наблюдений)

    Проводя т-тест, мы принимаем предположение о распределении наблюдений (=наших случайных величин) согласно распределению Стьюдента. Вместе с этим нам требуется подсчитать выборочное стандартное отклонение, которое может оказаться разным для двух тестов, и которое будет определять значение т-статистики и результаты теста.

    Соответсвенно и некие "истинные" средние распредлений в каждом из тестов тоже могут быть отличны.